In [1]:
import torch.distributions.distribution as D
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import data.read_samples as rs
import torch.optim as optim
import torch.utils.data
import torch.nn as nn
import numpy as np
import datetime
import torch
import time
import os

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 90
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

[INFO] Pre-processing for make clean
2022-05-14 03:27:06.692702 model.py code start


In [2]:
device = torch.device('cuda')
print(torch.cuda.get_device_name(device))
os.environ['KMP_DUPLICATE_LIB_OK']='True'

NVIDIA GeForce RTX 3080 Laptop GPU


In [3]:
class RBM(nn.Module): 
    with torch.cuda.device(0):
        def __init__(self, n_vis, n_hid, k, batch):
            super(RBM, self).__init__()
            self.W      = nn.Parameter(torch.randn(1, batch, device=device) * 1e-2)
            self.n_vis  = n_vis
            self.n_hid  = n_hid
            self.k      = k
            self.batch  = batch
            self.v_bias = nn.Parameter(torch.zeros(batch, device=device))
            self.h_bias = nn.Parameter(torch.zeros(batch, device=device))
        
        def sample_from_p(self, p):
            return F.relu(
                torch.sign(
                    p - Variable(torch.randn(p.size(), device=device))
                )
            ).to(device=device)

        ''' ISSUE PART '''
        def v_to_h(self, v):
            w = (self.W.clone())

            p_h = F.sigmoid(
                F.linear(v, w)
            ).to(device=device)

            sample_h = self.sample_from_p(p_h)
            return p_h, sample_h

        def h_to_v(self, h):
            w = self.W.t().clone()

            p_v = F.sigmoid(
                F.linear(h, w).to(device=device)
            ).to(device=device)

            sample_v = self.sample_from_p(p_v)
            return p_v, sample_v
        
        def forward(self, v):
            pre_h1, h1 = self.v_to_h(v)
            h_ = h1

            for _ in range(self.k):
                pre_v_, v_ = self.h_to_v(h_)
                pre_h_, h_ = self.v_to_h(v_)
            return v, v_
        
        def get_weight(self):
            return self.W

In [4]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
		 ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124']
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 100
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 102
[IWIP]		rdsamp Converting 103
[IWIP]		rdsamp Converting 104
[IWIP]		rdsamp Converting 105
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 107
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 111
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 113
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 117
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 121
[IWIP

In [5]:
train_dataloader = DataLoader(db1_sig + db2_sig,
                              batch_size=BATCH_SIZE,
                              num_workers=0, 
                              collate_fn=lambda x: x,
                              shuffle=True)
                              
test_dataloader = DataLoader(db2_sig + db3_sig,
                             batch_size=BATCH_SIZE,
                             num_workers=0, 
                             collate_fn=lambda x: x,
                             shuffle=True)

In [6]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)


first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

omse_loss = list()
mse_loss = nn.MSELoss()

In [7]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    start = time.time()
    '''First bbrbm'''
    for i, (data) in enumerate(train_dataloader):
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        # tensor binary
        vog_first, v1 = rbm_first(sample_data)
        omse_loss = mse_loss(vog_first, v1)

        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, omse_loss, time.time() - start))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2416/1758822892.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  torch.tensor(data, dtype=torch.float32)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1ST BBrbm_first Training loss for 0 epoch 0.5000889301300049	Estimate time : 6.622481822967529
1ST BBrbm_first Training loss for 1 epoch 0.5000483989715576	Estimate time : 4.993016242980957
1ST BBrbm_first Training loss for 2 epoch 0.5000117421150208	Estimate time : 4.791853427886963
1ST BBrbm_first Training loss for 3 epoch 0.49986883997917175	Estimate time : 4.858232259750366
1ST BBrbm_first Training loss for 4 epoch 0.5001017451286316	Estimate time : 10.673429012298584
1ST BBrbm_first Training loss for 5 epoch 0.49958404898643494	Estimate time : 5.0911865234375
1ST BBrbm_first Training loss for 6 epoch 0.4998660683631897	Estimate time : 5.035816669464111
1ST BBrbm_first Training loss for 7 epoch 0.5001747012138367	Estimate time : 4.573587656021118
1ST BBrbm_first Training loss for 8 epoch 0.500055193901062	Estimate time : 4.456308841705322
1ST BBrbm_first Training loss for 9 epoch 0.500137984752655	Estimate time : 4.499737024307251
1ST BBrbm_first Training loss for 10 epoch 0.500295

In [8]:
output_from_first = torch.tensor(output_from_first)
print(output_from_first.size())

for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    start = time.time()
    for _, (data) in enumerate(output_from_first): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        # tensor binary
        vog_second, v2 = rbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()

    end = time.time()
    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, omse_loss, end - start))


torch.Size([90, 1040000, 10])


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2416/2910812646.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)


2ST BBrbm_first Training loss for 0 epoch 0.4999169409275055	Estimate time : 16.507704973220825
2ST BBrbm_first Training loss for 1 epoch 0.49998509883880615	Estimate time : 15.797186374664307
2ST BBrbm_first Training loss for 2 epoch 0.4998663663864136	Estimate time : 14.3897123336792
2ST BBrbm_first Training loss for 3 epoch 0.49972376227378845	Estimate time : 14.305691242218018
2ST BBrbm_first Training loss for 4 epoch 0.4999684691429138	Estimate time : 14.107273817062378
2ST BBrbm_first Training loss for 5 epoch 0.5001271367073059	Estimate time : 14.257359027862549
2ST BBrbm_first Training loss for 6 epoch 0.49988222122192383	Estimate time : 14.21283483505249
2ST BBrbm_first Training loss for 7 epoch 0.5000804662704468	Estimate time : 14.246027708053589
2ST BBrbm_first Training loss for 8 epoch 0.5000073909759521	Estimate time : 14.498889446258545
2ST BBrbm_first Training loss for 9 epoch 0.4999373257160187	Estimate time : 14.56824541091919
2ST BBrbm_first Training loss for 10 epoc

In [9]:
output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third bbrbm'''
    for _, (data) in enumerate(output_from_second):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        vog_third, v3 = rbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()
    
    end = time.time()
    output_from_third.append(v3)
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, omse_loss, end - start))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2416/3297051644.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)


3ST BBrbm_first Training loss for 0 epoch 0.5001407861709595	Estimate time : 0.1400003433227539
3ST BBrbm_first Training loss for 1 epoch 0.49999797344207764	Estimate time : 0.1640019416809082
3ST BBrbm_first Training loss for 2 epoch 0.5000463724136353	Estimate time : 0.18200469017028809
3ST BBrbm_first Training loss for 3 epoch 0.4999676048755646	Estimate time : 0.1530003547668457
3ST BBrbm_first Training loss for 4 epoch 0.5000368356704712	Estimate time : 0.17299914360046387
3ST BBrbm_first Training loss for 5 epoch 0.4997451901435852	Estimate time : 0.15600013732910156
3ST BBrbm_first Training loss for 6 epoch 0.49995049834251404	Estimate time : 0.16013193130493164
3ST BBrbm_first Training loss for 7 epoch 0.5002955794334412	Estimate time : 0.1439952850341797
3ST BBrbm_first Training loss for 8 epoch 0.4996170401573181	Estimate time : 0.1530001163482666
3ST BBrbm_first Training loss for 9 epoch 0.49987635016441345	Estimate time : 0.16495728492736816
3ST BBrbm_first Training loss fo

In [10]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

output_from_first = list()
output_from_second = list()

omse_loss = list()
mse_loss = nn.MSELoss()
gaussian_std = torch.arange(1, 0, -0.1)

In [11]:
print(output_from_third)

[tensor([[0., 1., 1.,  ..., 1., 1., 0.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 0.],
        [0., 1., 0.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0',
       grad_fn=<ReluBackward0>), tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 0., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 0., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0',
       grad_fn=<ReluBackward0>), tensor([[0., 1., 1.,  ..., 0., 0., 1.],
        [1., 0., 1.,  ..., 1., 0., 1.],
        [1., 0., 1.,  ..., 1., 0., 1.],
        ...,
        [1., 1., 0.,  ..., 1., 1., 1.],
        [0., 1., 0.,  ..., 1., 0., 1.],
        [0., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0',
       grad_fn=<ReluBackward0>), tensor([[1., 1., 1.,  ..., 1., 1., 0.],
        [0., 1., 0.,  ..., 1., 1., 1.],
       

In [14]:
print(device)

cuda


In [17]:
loss_ = []
# output_from_third = torch.tensor(output_from_third)

for epoch in range(EPOCH):
    '''First gbrbm'''
    for i, (data) in enumerate(output_from_third):
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.normal(mean=data, std=gaussian_std).view(-1, 10)

        # tensor binary
        vog_first, v1 = rbm_first(sample_data)
        omse_loss = mse_loss(vog_first, v1)

        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()
    output_from_first.append(v1.tolist())
    print("1ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, omse_loss, mt))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2416/3449405752.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)


TypeError: normal() received an invalid combination of arguments - got (mean=Tensor, std=Tensor, device=torch.device, ), but expected one of:
 * (Tensor mean, Tensor std, *, torch.Generator generator, Tensor out)
 * (Tensor mean, float std, *, torch.Generator generator, Tensor out)
 * (float mean, Tensor std, *, torch.Generator generator, Tensor out)
 * (float mean, float std, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
output_from_first = torch.tensor(output_from_first)
output_from_third = list()
print(output_from_first.size())

for epoch in range(EPOCH):
    '''Secnd gbrbm'''
    start = time.time()
    for _, (data) in enumerate(output_from_first): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.normal(mean=data, std=gaussian_std).view(-1, 10)

        # tensor binary
        vog_second, v2 = rbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()

    end = time.time()
    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, omse_loss, end - start))

In [ ]:
output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third gbrbm'''
    for _, (data) in enumerate(output_from_second):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10)

        vog_third, v3 = rbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()
    
    end = time.time()
    output_from_third.append(v3)
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, omse_loss, end - start))
